# problematic and goal

In order to train a CNN, the shape of the input should be equal for all samples.
the width and height are 512x512 for each slice but the number of slices differs.
We are going to create samples with equal number of slices.

# installing the requirements

In [1]:
# %pip install glob2
# %pip install pytest-shutil

In [2]:
from glob import glob # directory exploration
import shutil # managing files
import os # managing paths

# Creating groups of 64 slices

In [3]:
in_path = 'datasets/dicom_files/images' # created by 3dslicer after processing nifti files
out_path = 'datasets/dicom_groups/images'

In [4]:
for patient in glob(in_path + '/*'):
    patient_name = os.path.basename(os.path.normpath(patient))
    number_folders = int(len(glob(patient+'/*'))/64)

    for i in range(number_folders):
        out_path_name = os.path.join(out_path, patient_name+'_'+str(i))
        os.mkdir(out_path_name)
        for i, file in enumerate(glob(patient+'/*')):
            if i == 64:
                break
            shutil.move(file, out_path_name)


# Convert the dicom files into nifties

In [5]:
%pip install dicom2nifti

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 3.3 MB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 16.8 MB 975 kB/s eta 0:00:01
  Using cached scipy-1.7.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (39.8 MB)
     |████████████████████████████████| 2.0 MB 724 kB/s eta 0:00:01
  Using cached packaging-21.3-py3-none-any.whl (40 kB)
     |████████████████████████████████| 98 kB 1.3 MB/s eta 0:00:01
Using legacy 'setup.py install' for dicom2nifti, since package 'wheel' is not installed.
    Running setup.py install for dicom2nifti ... done
You should consider upgrading via the '/usr/sbin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import dicom2nifti

In [7]:
in_path_images = 'datasets/dicom_groups/images'
in_path_labels = 'datasets/dicom_groups/labels'

out_path_images = 'datasets/nifti_files/images'
out_path_labels = 'datasets/nifti_files/labels'

In [8]:
list_images = glob(in_path_images)
list_labels = glob(in_path_labels)

In [9]:
for patient in list_images:
    patient_name = os.path.basename(os.path.normpath(patient))
    dicom2nifti.dicom_series_to_nifti(patient, os.path.join(out_path_images, patient_name + '.nii.gz'))

In [10]:
for patient in list_labels:
    patient_name = os.path.basename(os.path.normpath(patient))
    dicom2nifti.dicom_series_to_nifti(patient, os.path.join(out_path_labels, patient_name + '.nii.gz'))